In [50]:
import pandas as pd
import matplotlib.pyplot as ply
import seaborn as sns
import numpy as np
import scipy.stats as st
import statsmodels.formula.api as smf

In [51]:
import pandas as pd
import matplotlib.pyplot as ply
import seaborn as sns
import numpy as np
import scipy.stats as st
# Load libraries
import pandas as pd
#import numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
import statsmodels.formula.api as smf
from sklearn import linear_model
from patsy.builtins import Q
from sklearn.linear_model import RidgeCV

In [52]:
df = pd.read_csv("LoanStats3b.csv",low_memory =False)

In [53]:
#df.head(10)


In [54]:
print('Number of missing values per column:')
countMissing = df.isnull().sum()
print(countMissing)

Number of missing values per column:
id                                            188181
member_id                                     188181
loan_amnt                                          0
funded_amnt                                        0
funded_amnt_inv                                    0
term                                               0
int_rate                                           0
installment                                        0
grade                                              0
sub_grade                                          0
emp_title                                      11737
emp_length                                      7887
home_ownership                                     0
annual_inc                                         0
verification_status                                0
issue_d                                            0
loan_status                                        0
pymnt_plan                                         0
url      

In [55]:
# Drop the columns where all elements are missing
#df = df.dropna(axis=0,how='any')
df = df.dropna(axis=1,how='all')


In [56]:
# Columns that are being dropped
df=df.drop(['hardship_dpd','hardship_loan_status','emp_title','hardship_type','hardship_reason','hardship_status','deferral_term','hardship_amount'],axis=1)
df=df.drop(['settlement_status','settlement_date','settlement_amount','settlement_term', 'policy_code', 'acc_now_delinq', 'num_tl_30dpd'],axis=1)
df=df.drop(['hardship_start_date','hardship_end_date','orig_projected_additional_accrued_interest','hardship_payoff_balance_amount','debt_settlement_flag_date'],axis=1)
df=df.drop(['total_bal_ex_mort', 'num_sats', 'tot_cur_bal', 'tax_liens', 'zip_code','addr_state','title', 'num_tl_90g_dpd_24m'], axis=1)
df=df.drop(['payment_plan_start_date','hardship_length','hardship_last_payment_amount','settlement_percentage', 'collections_12_mths_ex_med'],axis=1)
df=df.drop(['mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'mths_since_last_delinq' ,'mths_since_last_major_derog'],axis=1)
df=df.drop(['hardship_flag', 'debt_settlement_flag', 'num_tl_120dpd_2m', 'chargeoff_within_12_mths', 'delinq_amnt', 'application_type', 'emp_length'],axis=1)
df=df.drop(['out_prncp', 'out_prncp_inv'], axis=1)

In [57]:
print('Number of missing values per column:')
countMissing = df.isnull().sum()
print(countMissing)

Number of missing values per column:
loan_amnt                          0
funded_amnt                        0
funded_amnt_inv                    0
term                               0
int_rate                           0
installment                        0
grade                              0
sub_grade                          0
home_ownership                     0
annual_inc                         0
verification_status                0
issue_d                            0
loan_status                        0
pymnt_plan                         0
desc                          106703
purpose                            0
dti                                0
delinq_2yrs                        0
earliest_cr_line                   0
inq_last_6mths                     0
open_acc                           0
pub_rec                            0
revol_bal                          0
revol_util                       125
total_acc                          0
initial_list_status                0
t

In [58]:
# df = df.drop(df[df['revol_util']==0].index)
# df.drop(df.index[df['last_credit_pull_d'] == 0], inplace = True)

In [59]:
df['int_rate'] = df['int_rate'].str.rstrip('%')
df['int_rate'] = df['int_rate'].astype('float64')

In [60]:
numericalList = []
nonNumList = []
for column in df.columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
        numericalList.append(column)
    else:
        nonNumList.append(column)

In [61]:
numDF = df[numericalList]
nonnumDF = df[nonNumList]

In [62]:
nrow = len(numDF['int_rate'])
# print('Number of missing values per column:')
num_countMissing = numDF.isnull().sum()
# print(num_countMissing)

In [63]:
# Estimate the mean for columns that are missing less than 20% of the observations
for coln in numDF.columns:
    if num_countMissing[coln] != 0:
        if num_countMissing[coln]/nrow < 0.20:
            numDF[coln] = numDF[coln].fillna(value=numDF[coln].mean())

/Users/michellebaginski/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [64]:
# remove all the observations with NAs in them
df = df.dropna(axis=0,how='any')

print('Number of missing values per column:')
countMissing = df.isnull().sum()
print(countMissing)

Number of missing values per column:
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
int_rate                      0
installment                   0
grade                         0
sub_grade                     0
home_ownership                0
annual_inc                    0
verification_status           0
issue_d                       0
loan_status                   0
pymnt_plan                    0
desc                          0
purpose                       0
dti                           0
delinq_2yrs                   0
earliest_cr_line              0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
initial_list_status           0
total_pymnt                   0
total_pymnt_inv               0
total_rec_prncp               0
total_rec_int                 0
   

In [65]:
# Removing variables on the basis of correlation matrix
corr = numDF.corr()
print(corr)
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.8:
            if columns[j]:
                columns[j] = False
selected_columns = numDF.columns[columns]


                            loan_amnt  funded_amnt  funded_amnt_inv  int_rate  \
loan_amnt                    1.000000     0.999799         0.999663  0.182654   
funded_amnt                  0.999799     1.000000         0.999874  0.182485   
funded_amnt_inv              0.999663     0.999874         1.000000  0.182933   
int_rate                     0.182654     0.182485         0.182933  1.000000   
installment                  0.955011     0.955254         0.955211  0.165173   
annual_inc                   0.368164     0.368151         0.368084 -0.026026   
dti                          0.044557     0.044572         0.044746  0.147471   
delinq_2yrs                  0.011184     0.011214         0.011391  0.097230   
inq_last_6mths               0.019741     0.019703         0.020091  0.241345   
open_acc                     0.191571     0.191614         0.191719  0.017359   
pub_rec                     -0.073588    -0.073537        -0.073373  0.056575   
revol_bal                   

In [66]:
numDF = numDF[selected_columns]



In [67]:
# £print(nonnumDF[['grade','home_ownership','loan_status','purpose','sub_grade','term']])

In [68]:
# Quartiles and IQR for mths_since_recent_bc

for col in numDF.columns:
    quartiles = numDF[col].quantile([0.25, 0.75], interpolation='nearest')
    q1 = quartiles[0.25]
    q3 = quartiles[0.75]
    IQR = q3 - q1
    outlier_val = q3 + 1.5*IQR
    #print("Outlier val:", outlier_val)
    numDF[col] = np.where(numDF[col] > outlier_val, outlier_val, numDF[col])

#df1 = df['mths_since_recent_bc'] = np.where(df['mths_since_recent_bc'] > outlier_val, outlier_val, df['mths_since_recent_bc'])

In [69]:
nonnumDF.columns

Index(['term', 'grade', 'sub_grade', 'home_ownership', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'desc', 'purpose',
       'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d',
       'last_credit_pull_d'],
      dtype='object')

In [70]:
finaldf = pd.concat([numDF, nonnumDF[['grade','home_ownership','loan_status','purpose','term','sub_grade','verification_status']]], axis=1)

In [71]:
finaldf_mn = pd.concat([numDF, nonnumDF[['grade','home_ownership','loan_status','purpose','term','sub_grade','verification_status']]], axis=1)

In [72]:
np.unique(finaldf['loan_status'])

array(['Charged Off', 'Fully Paid'], dtype=object)

In [73]:
finaldf["loan_status"] = np.where(finaldf["loan_status"] == "Charged Off", 1, 0)

In [74]:
finaldf_mn["loan_status"] = np.where(finaldf_mn["loan_status"] == "Charged Off", 1, 0)


In [75]:
print('Number of missing values per column:')
countMissing = finaldf_mn['loan_status']
count = 0
ncount = 0
for w in countMissing:
    if w == 1:
        count+=1
    else:
        ncount+=1
print(count/len(finaldf_mn['loan_status']))
print(ncount/len(finaldf_mn['loan_status']))

Number of missing values per column:
0.15767266620965986
0.8423273337903402


In [76]:
## Undersampling because of imbalanced dataset
Chargedoff = len(finaldf_mn[finaldf_mn['loan_status'] == 1])
Paid_indices = finaldf_mn[finaldf_mn.loan_status == 0].index
random_indices = np.random.choice(Paid_indices,Chargedoff, replace=False)
Chargedoff_indices = finaldf_mn[finaldf_mn.loan_status == 1].index
under_sample_indices = np.concatenate([Chargedoff_indices,random_indices])
finaldf_mn = finaldf_mn.loc[under_sample_indices]
# finaldf_main['loan_status']
#print(Chargedoff)

In [77]:
## Undersampling because of imbalanced dataset
Chargedoff = len(finaldf[finaldf['loan_status'] == 1])
Paid_indices = finaldf[finaldf.loan_status == 0].index
random_indices = np.random.choice(Paid_indices,Chargedoff, replace=False)
Chargedoff_indices = finaldf[finaldf.loan_status == 1].index
under_sample_indices = np.concatenate([Chargedoff_indices,random_indices])
finaldf = finaldf.loc[under_sample_indices]
# finaldf['loan_status']
#print(Chargedoff)

In [78]:
# Selecting categorical variables based on chi2 test
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
finaldf['grade']  = finaldf['grade'].astype('str')
finaldf['home_ownership']  = finaldf['home_ownership'].astype('str')
finaldf['purpose']  = finaldf['purpose'].astype('str')
finaldf['term']  = finaldf['term'].astype('str')
finaldf['sub_grade']  = finaldf['sub_grade'].astype('str')
finaldf['verification_status']  = finaldf['verification_status'].astype('str')


finaldf['grade'] = label_encoder.fit_transform(finaldf['grade'])
finaldf['home_ownership'] = label_encoder.fit_transform(finaldf['home_ownership'])
finaldf['purpose'] = label_encoder.fit_transform(finaldf['purpose'])
finaldf['term'] = label_encoder.fit_transform(finaldf['term'])
finaldf['sub_grade'] = label_encoder.fit_transform(finaldf['sub_grade'])
finaldf['verification_status'] = label_encoder.fit_transform(finaldf['verification_status'])

In [79]:
from sklearn.feature_selection import chi2
chi_scores = chi2(finaldf[['grade','home_ownership','purpose','term','sub_grade','verification_status']],finaldf['loan_status'])

In [80]:
chi_scores

(array([ 3916.51881998,   321.17133001,   248.92036159,  1600.00571755,
        17191.24455441,   323.46331733]),
 array([0.00000000e+00, 8.04936533e-72, 4.46490874e-56, 0.00000000e+00,
        0.00000000e+00, 2.54992977e-72]))

In [81]:
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.preprocessing import LabelEncoder

predictors = ['grade','home_ownership','purpose','term','sub_grade','verification_status']


# Perform feature selection
selector = SelectKBest(chi2, k='all')
selector.fit(finaldf, finaldf['loan_status'])

# Get the raw p-values for each feature, and transform from p-values into scores
scores = selector.scores_
print(scores)
# Plot the scores.  See how "Pclass", "Sex", "Title", and "Fare" are the best?
# plt.bar(range(len(predictors)), scores)
# plt.xticks(range(len(predictors)), predictors, rotation='vertical')
# plt.show()

[1.64254588e+06 6.38815215e+03 8.26039419e+06 2.35091351e+03
            nan 4.29675327e+02 4.89342920e+01            nan
 8.48236435e+04 9.27092466e+01            nan            nan
 9.23950242e+07            nan 1.99136345e+06 1.02669607e+03
 3.45998425e+06 4.93849023e+06 4.26101196e+03 7.86569796e+02
 5.04817503e+03 1.97804989e+03 1.37252174e+03 4.36354981e+02
 5.62182132e+03 1.12787413e+03 4.54951947e+00 5.48857982e+01
 3.03663345e+02 9.02943303e+00 1.10957385e+01 9.53377200e+01
 3.67138267e+02 2.34375085e+00            nan 1.22098862e+05
 3.91651882e+03 3.21171330e+02 2.96710000e+04 2.48920362e+02
 1.60000572e+03 1.71912446e+04 3.23463317e+02]


In [33]:
## Dividing dataset into 2 parts
x_train, x_test, y_train, y_test = train_test_split(finaldf_mn.loc[:, finaldf_mn.columns != 'loan_status'], finaldf_mn['loan_status'], test_size=0.3, random_state = 0)


In [82]:
x_train_sc = x_train[['loan_amnt', 'int_rate', 'annual_inc', 'dti','revol_bal','last_pymnt_amnt','total_rev_hi_lim','avg_cur_bal',
       'bc_open_to_buy', 'bc_util']]
x_test_sc = x_test[['loan_amnt', 'int_rate', 'annual_inc', 'dti','revol_bal','last_pymnt_amnt','total_rev_hi_lim','avg_cur_bal',
       'bc_open_to_buy', 'bc_util']]

In [83]:
scaler = StandardScaler()
scaler.fit(x_train_sc)
scaler.fit(x_test_sc)
names = x_train_sc.columns
x_train_scaled = scaler.transform(x_train_sc)
x_train_scaled = pd.DataFrame(x_train_scaled, columns=names)
x_test_scaled = scaler.transform(x_test_sc)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=names)

In [84]:
x_train

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,...,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,total_il_high_credit_limit,grade,home_ownership,purpose,term,sub_grade,verification_status
0,11075.0,13.11,32000.0,17.93,0.0,0.0,10.0,0.0,6946.0,18.0,...,1.000000,100.000000,0.0,12398.000000,B,MORTGAGE,debt_consolidation,36 months,B4,Source Verified
1,21000.0,12.12,100000.0,17.74,0.0,0.0,12.0,0.0,12811.0,33.0,...,1.000000,97.000000,0.0,59807.000000,B,MORTGAGE,debt_consolidation,60 months,B3,Verified
2,2650.0,16.78,22800.0,12.00,0.0,1.0,12.0,0.0,9265.0,49.0,...,1.000000,98.000000,0.0,0.000000,C,OWN,other,36 months,C5,Verified
3,4000.0,13.68,41000.0,22.62,0.0,2.5,14.0,0.0,9828.0,29.0,...,3.000000,100.000000,0.0,31793.000000,C,RENT,debt_consolidation,36 months,C1,Not Verified
4,8025.0,23.28,37235.0,23.11,0.0,2.0,10.0,0.0,4997.0,18.0,...,1.000000,88.000000,0.0,23403.000000,F,RENT,debt_consolidation,60 months,F2,Verified
5,2350.0,15.81,30000.0,29.48,0.0,2.0,11.0,0.0,3209.0,11.0,...,1.788014,95.399571,0.0,34389.556102,C,RENT,other,36 months,C4,Not Verified
6,5000.0,18.75,54000.0,15.98,0.0,2.0,5.0,0.0,14261.0,22.0,...,1.000000,100.000000,0.0,10264.000000,D,RENT,debt_consolidation,36 months,D3,Not Verified
7,12800.0,13.99,65000.0,14.86,0.0,2.0,14.0,0.0,24294.0,30.0,...,1.788014,95.399571,0.0,34389.556102,C,MORTGAGE,credit_card,36 months,C1,Not Verified
8,2000.0,17.27,25000.0,24.41,0.0,0.0,5.0,0.0,13608.0,9.0,...,0.000000,100.000000,0.0,0.000000,C,RENT,debt_consolidation,36 months,C5,Not Verified
9,5375.0,9.71,41000.0,19.40,0.0,0.0,8.0,0.0,2333.0,11.0,...,0.000000,80.000000,0.0,31856.000000,B,RENT,debt_consolidation,36 months,B1,Verified


In [85]:
x_train=x_train.reset_index()
#x_train = x_train.drop('index')
x_train = x_train.loc[:, x_train.columns != 'index']

In [86]:
x_test=x_test.reset_index()
#x_train = x_train.drop('index')
x_test = x_test.loc[:, x_test.columns != 'index']

In [87]:
x_train_scaled = pd.concat([x_train_scaled[['loan_amnt', 'int_rate', 'annual_inc', 'dti','revol_bal','last_pymnt_amnt','total_rev_hi_lim','avg_cur_bal',
       'bc_open_to_buy', 'bc_util']], x_train[['sub_grade','home_ownership','purpose','term']]], axis=1)

In [88]:
x_test_scaled = pd.concat([x_test_scaled[['loan_amnt', 'int_rate', 'annual_inc', 'dti','revol_bal','last_pymnt_amnt','total_rev_hi_lim','avg_cur_bal',
       'bc_open_to_buy', 'bc_util']], x_test[['sub_grade','home_ownership','purpose','term']]], axis=1)

In [89]:
#making categorical variable dummy variable 
x_train_scaled = pd.get_dummies(x_train_scaled, columns=['home_ownership','purpose','term','sub_grade'])


In [90]:
#making categorical variable dummy variable 
x_test_scaled = pd.get_dummies(x_test_scaled, columns=['home_ownership','purpose','term','sub_grade'])


In [91]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(x_train_scaled,y_train)
# predictions
rfc_predict = rfc.predict(x_test_scaled)

/Users/michellebaginski/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [92]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [93]:
rfc_cv_score = cross_val_score(rfc, x_train_scaled, y_train, cv=10, scoring='roc_auc')

In [94]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[6808 2148]
 [1859 6988]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.79      0.76      0.77      8956
           1       0.76      0.79      0.78      8847

    accuracy                           0.77     17803
   macro avg       0.78      0.78      0.77     17803
weighted avg       0.78      0.77      0.77     17803



=== All AUC Scores ===
[0.86952638 0.86273777 0.8465039  0.86014253 0.84162513 0.85273836
 0.85622041 0.86176969 0.86549607 0.86477387]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8581534094395709


In [95]:
rfc = RandomForestClassifier(n_estimators=600, max_depth=300, max_features='sqrt')
rfc.fit(x_train_scaled,y_train)
rfc_predict = rfc.predict(x_test_scaled)
rfc_cv_score = cross_val_score(rfc, x_train_scaled, y_train, cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[6439 2517]
 [ 941 7906]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.87      0.72      0.79      8956
           1       0.76      0.89      0.82      8847

    accuracy                           0.81     17803
   macro avg       0.82      0.81      0.80     17803
weighted avg       0.82      0.81      0.80     17803



=== All AUC Scores ===
[0.88882677 0.88471159 0.87540674 0.8839003  0.86899906 0.87545567
 0.88477481 0.8874521  0.89123241 0.88543022]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.882618967205401


In [96]:
for name, importance in zip(x_train_scaled, rfc.feature_importances_):
    print(name, "=", importance)

loan_amnt = 0.053761735402326115
int_rate = 0.06534370526283916
annual_inc = 0.054183461727034365
dti = 0.058000113452690065
revol_bal = 0.05403827061955187
last_pymnt_amnt = 0.3890238634759968
total_rev_hi_lim = 0.04660280187282706
avg_cur_bal = 0.04980539790538859
bc_open_to_buy = 0.05199553285759633
bc_util = 0.0525780966143542
home_ownership_MORTGAGE = 0.006340654698363726
home_ownership_NONE = 0.00010530702127745545
home_ownership_OTHER = 9.771610627428151e-05
home_ownership_OWN = 0.0043011820447871885
home_ownership_RENT = 0.00627023612486046
purpose_car = 0.0012395108156892766
purpose_credit_card = 0.006003184493149518
purpose_debt_consolidation = 0.007169357383117477
purpose_home_improvement = 0.0032070390840867975
purpose_house = 0.0007942808990084938
purpose_major_purchase = 0.001858942628901374
purpose_medical = 0.0011033730401453033
purpose_moving = 0.0008226783482957738
purpose_other = 0.003211154227492836
purpose_renewable_energy = 0.00010885846394341059
purpose_small_bus